In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

# A/B testing to improve Conversion Rates

You work for Company K. Company K is a video streaming service that is selling subscriptions. After running a few analyses, you suggest to the Product Team that placing more prominent upsells on the landing page may result in higher conversion rates (visitors signing up for the subscription). The Product Team took your suggestion with excitement and the engineering team went ahead and implemented the experiment right away. 

The experiment ran for the month of February 2020. You prepared a few datasets beforehand that will help you in analyzing the success of the A/B test.
The A/B test had two experiment groups, A & B, in which visitors assigned to A were in the control group and visitors assigned to B received the new variant with the new upsells. You suggest to assign approximately 7% of the daily visitors to the experiment, applying a 50/50 split among the two variants.

You have the following datasets:

1. `first_pageviews`: This is a summary of all the unsubscribed users and the timestamp of their first pageview. Users get assigned according to their cookies, which is how we make sure that a user may not be assigned to two different groups when they reach our landing page. Since visitors haven't signed up at this point, they don't have a `user_id`, instead, we assign them a random but unique `anonymous_id`. If visitors get assigned to an experiment, you should see a non-null value in the `exp_id` column. Your experiment has the ids **101** for control and **102** for the variant.
2. `subscribers`: This table shows you which users have subscribed at what point in time. To subscribe, you need to be a registered user, which is how you can uniquely identify each subscriber. Since we only sell yearly subscriptions, users cannot have more than 1 subscription within 365 days. 
3. `users`: This table is useful to link the first pageview of unsubscribed visitors to our subscribers. It keeps track of the `anonymous_id` we assigned to them and maps to the `user_id` we assigned after they sign up

Immediately after the end of the experiment, the Product Manager approaches you overly excited and wants you to analyze the success of the experiment.

You can find the datasets in the `data/2020-03-01` folder.

In [3]:
# load the datasets
first_pageviews = pd.read_csv("data/2020-03-01/first_pageviews.csv")
users = pd.read_csv("data/2020-03-01/users_mar01.csv")
subscriptions = pd.read_csv("data/2020-03-01/subscriptions_mar01.csv")

## Inspect the data

In [4]:
# show the first 5 lines of each dataset
first_pageviews.head()

,date_id,anonymous_id,exp_id
0,2020-01-16,4d005f94-e06f-493c-b569-9efb5ce4d203,NaN
1,2020-01-16,948ad58a-f483-4007-aa56-280aa018427b,NaN
2,2020-01-16,31f4f2e4-4ae6-417b-8a6d-53b98a719145,NaN
3,2020-01-16,24747dc7-355a-44b0-95c6-0caa91cd4ae4,NaN
4,2020-01-16,0162c4f8-7bc2-4432-90b9-bfb61f5e57b3,NaN


In [5]:
users.head()

,anonymous_id,user_id
0,e575dd8c-f02c-4a1a-8a53-fef7ff2b9f95,b67781da-3a0f-48af-80d8-728d3ec31c66
1,a92532b2-0317-49bb-9f00-3d9b9ae71ea5,ce1764ca-91a2-4c2e-95fe-c415c5df7be5
2,9c39b4d4-d7da-49e2-bcd9-4abee3e211f6,b24712a4-40b7-4841-a9e9-f566d9ed48a9
3,858cc4cc-dcd2-4897-815b-500a2ee8c00e,d0e4f27e-11ab-470c-85db-4f7389ba63d4
4,96a9ad69-568e-40bd-9ff7-3e6b8e0d3f9a,91b679ad-38cc-4569-b5fc-8a98dc858b8d


In [6]:
subscriptions.head()

,user_id,created_at,client_app
0,b67781da-3a0f-48af-80d8-728d3ec31c66,2020-01-26 09:00:55.628471160,android
1,ce1764ca-91a2-4c2e-95fe-c415c5df7be5,2020-01-30 10:43:16.359589080,ios
2,b24712a4-40b7-4841-a9e9-f566d9ed48a9,2020-02-15 21:19:35.663792880,web
3,d0e4f27e-11ab-470c-85db-4f7389ba63d4,2020-02-13 04:45:52.198410780,ios
4,91b679ad-38cc-4569-b5fc-8a98dc858b8d,2020-02-10 13:24:29.670026400,android


In [ ]:
# check the data types and set them properly for each dataset

In [7]:
first_pageviews.dtypes

date_id          object
anonymous_id     object
exp_id          float64
dtype: object

In [8]:
users.dtypes

anonymous_id    object
user_id         object
dtype: object

In [9]:
subscriptions.dtypes

user_id       object
created_at    object
client_app    object
dtype: object

In [11]:
# plot first pageview per day to inspect if you can spot any outliers or anything suspicious
first_pageviews["number"]=1
ppd = first_pageviews.groupby("date_id").agg({"number":"sum"})
ppd.head()

,number
date_id,
2020-01-16,43732
2020-01-17,46950
2020-01-18,32288
2020-01-19,34464
2020-01-20,52184


In [17]:
fix, ax = plt.subplots(figsize=(10,6))

sns.distplot(ppd, bins=15)

In [29]:
# check the assignment into the experiment. You should see that 7% of the visitors were 
# assigned to the experiment with a 50/50 split among the two variants
groups = first_pageviews.groupby("exp_id").agg({"number":"sum"})
groups["number"]/len(first_pageviews)

exp_id
101.0    0.034329
102.0    0.034242
Name: number, dtype: float64

The assignment seemed to work as planned. You can now continue with the data prep.

## Create experiment dataset

To simplify the analysis, filter and merge the datasets to only focus on what's relevant. That is, you only care about visitors that were actually assigned to the experiment. Among those, you want to know how many subscribed. The proportion of those, for each group, is the conversion rate.

In [145]:
# merge the tables so you can see all visitors assigned to the experiment 
# and whether they subscribed or not
# create a dummy column for whether a user subscribed or not
exp = pd.merge(first_pageviews,users,how="left", left_on="anonymous_id", right_on="anonymous_id")
exp = pd.merge(exp, subscriptions,how="left", left_on="user_id", right_on="user_id")

In [146]:
exp.head()

,date_id,anonymous_id,exp_id,number,user_id,created_at,client_app
0,2020-01-16,4d005f94-e06f-493c-b569-9efb5ce4d203,NaN,1,NaN,NaN,NaN
1,2020-01-16,948ad58a-f483-4007-aa56-280aa018427b,NaN,1,NaN,NaN,NaN
2,2020-01-16,31f4f2e4-4ae6-417b-8a6d-53b98a719145,NaN,1,NaN,NaN,NaN
3,2020-01-16,24747dc7-355a-44b0-95c6-0caa91cd4ae4,NaN,1,NaN,NaN,NaN
4,2020-01-16,0162c4f8-7bc2-4432-90b9-bfb61f5e57b3,NaN,1,NaN,NaN,NaN


## Inspect experiment data

Before running the test, you want to spend some time checking the data you collected.

In [147]:
exp["date_id"] = pd.to_datetime(exp["date_id"])

In [148]:
# plot a timeseries of daily visitors that were assigned to an experiment
# plot a line for each exp_id
import datetime as dt
exp=exp.groupby(["date_id", "exp_id"]).agg({"number":"sum"}).reset_index()


In [149]:
exp["first_date"]=exp.iloc[0,0]

In [150]:
exp["days"]=(exp["date_id"]-exp["first_date"]).dt.days

In [151]:
f, ax = plt.subplots(figsize=(15,9))

ax = sns.lineplot(data=exp[exp["exp_id"]==101.0],x="days", y="number", color="b");
ax = sns.lineplot(data=exp[exp["exp_id"]==102.0],x="days", y="number", color="r");

In [ ]:
# does the time range match the experiment requirements?
# Do the lines indicate an equal assignment over time?

## Describing the data

Compute the differences among the groups, before running a statistical test

In [180]:
conversion = pd.merge(first_pageviews,users,how="left", left_on="anonymous_id", right_on="anonymous_id")
conversion = pd.merge(conversion, subscriptions,how="left", left_on="user_id", right_on="user_id")

In [181]:
conversion["date_id"] = pd.to_datetime(conversion["date_id"])

In [182]:
conversion["created_at"] = pd.to_datetime(conversion["created_at"]).dt.normalize()

In [186]:
conversion=conversion.assign(subscriber = np.where(conversion["user_id"].isna(),0,1))

In [137]:
# compute the number of conversion for each experiment group
conv_101 = conversion[conversion["exp_id"]==101.0]
conv_102 = conversion[conversion["exp_id"]==102.0]

In [ ]:
# compute the conversion rate for each experiment group ---> Absolute number, for percent, 

In [159]:
rate_101 = len(conv_101[conv_101["created_at"].between("2020-02-01","2020-02-29")])/len(conv_101)

In [160]:
rate_102 = len(conv_102[conv_102["created_at"].between("2020-02-01","2020-02-29")])/len(conv_102)

In [152]:
rate_101

0.007262067848041598

In [153]:
rate_102

0.007900760540500628

In [ ]:
# compute the conversion rate for each experiment group

In [179]:
# interpret the results. From just looking at the numbers, which group performed better 
# with respect to conversions or conversion rates?


You should see that there is a difference in the conversion rate across the different experiment groups.

Are there any differences among the subscibers for each group with respect to the client application they signed up through?

In [195]:
# Compute relative frequency for each client_app through which they subscribed in each experiment group
conv_client = conversion.copy()
conv_client = conv_client.groupby(["exp_id", "subscriber"]).agg({"number":"sum"})
conv_client


#cr_total = (conversion
#            .groupby(["exp_id", "subscriber"])
#            .agg({"anonymous_id": "count"})
#            .rename(columns={"anonymous_id": "visitors"}))
#cr_total

number
exp_id subscriber        
101.0  0            67394
       1              493
102.0  0            67180
       1              535

In [ ]:
# interpret and discuss the results

## Test whether the difference in conversion rate is significant

It is time for the statistical test!

In [ ]:
# compute the two arrays/series to be passed to the t-test

In [ ]:
# run the test and compute the p-value, without the equal variance assumption

In [ ]:
# Write up your conclusion and discuss

The experiment unfortunately did not result in a significant difference among the two groups. That is, you cannot conclude that the upsells were in fact improving the conversion rates. You still doubt whether you've done everything in the right way. 

During a coffee break, you overheard some colleague talking about how they are contemplating about getting newspaper subscription. They said they have been going to the newspaper website a few times already but they're not sure if they're willing to pay the price. You start thinking that you were maybe a little to rushed with starting the analysis on the first day after the experiment ended. It might take some time for people to subscribe after the first pageview...

In [ ]:
# check the latest date a subscription was registered

In [ ]:
# Using the Five-number summary, check how long it takes for users to subscribe after their first visit

In [ ]:
# What is flawed when computing the five-number summary using this sample?

## Considering time it takes to subscribe

The team decides to wait for the analysis until the end of the month to allow users to subscribe. On April 1st, the Product Manager approaches you with even more excitement and pushes you to redo the analysis.

The data for the additional subscribers that signed up between the day of your first analysis and today can be found in `data/2020-04-01/`.

**Important:** The new data does not include any of the data that were included in the previous datasets.

In [ ]:
# load the newest data for subscriptions and their corresponding mapping in the users dataset

In [ ]:
# combine the newest data with the existing datasets

In [ ]:
# From here on, you're on your own. Redo the analysis from above with the new dataset.
# Make sure to run a few inspections to make sure your data is how you expect it to be
# After analyzing, provide a summary and a discussion including your communication to the Product Team